In [ ]:
#!pip install transformers bitsandbytes peft accelerate trl

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(prompt):
    return tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
    )

from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    # Specify target modules as needed
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    # Include other relevant training arguments
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Your prepared dataset
    eval_dataset=eval_dataset    # Your evaluation dataset
)
